# Read Functions

Continuation of the previous notebook. Will modularize the read components into functions

In [ ]:
import os
import time
import glob
import struct
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

file = os.path.expanduser('~/Downloads/20200614_150015.256007-87-02.iqData.XXXX.AKITA.dat')
# file = os.path.expanduser('~/Downloads/20200615_010423.194421-8C-02.iqData.XXXX.AKITA.dat')
filesize = os.path.getsize(file)

plt.style.use('./darkmode.style')

In [ ]:
class Object(object):
    pass

In [ ]:
def read_file(file, verbose=0):
    # Some pre-defined structure
    head_struct = struct.Struct('IIIIHH')
    pack_struct = struct.Struct('IIIIIHHI')
    cpi_struct = struct.Struct('IIIIHHHHHHHHHHIII')
    pri_struct = struct.Struct('IIHHHH')

    # Pulse
    def read_pulse(fid, pri_struct, cpi_header):
        tmp = fid.read(32 * 4)
        n = pri_struct.unpack_from(tmp)
        if n[0] != 0x11111111 or n[1] != 0xEEEEEEEE:
            print('Error: Incorrect check codes: 0x{:08X} 0x{:08X}'.format(n[0], n[1]))
            return None

        # I/Q data
        def read_iq_block(align_num, num, fid):
            if align_num:
                tmp = fid.read(align_num * 4)
                raw = np.frombuffer(tmp, dtype='<i2', count=num*2)
                return np.array(raw[::2] + 1j * raw[1::2], dtype=np.csingle)
            else:
                return np.empty(0)

        # Pulse
        pulse = Object()
        pulse.azimuth = (n[4] & 0x3FFF) * 360 / 2 ** 14
        pulse.elevation = (n[5] & 0x3FFF) * 180 / 2 ** 13
        pulse.ngate_long_hi = cpi_header.num_range_long_hi
        pulse.ngate_long_lo = cpi_header.num_range_long_lo
        pulse.ngate_short_hi = cpi_header.num_range_short_hi
        pulse.ngate_short_lo = cpi_header.num_range_short_lo

        # H channel data
        pulse.h_long_hi = read_iq_block(cpi_header.align_num_range_long_hi, cpi_header.num_range_long_hi, fid)
        pulse.h_long_lo = read_iq_block(cpi_header.align_num_range_long_lo, cpi_header.num_range_long_lo, fid)
        pulse.h_short_hi = read_iq_block(cpi_header.align_num_range_short_hi, cpi_header.num_range_short_hi, fid)
        pulse.h_short_lo = read_iq_block(cpi_header.align_num_range_short_lo, cpi_header.num_range_short_lo, fid)

        # V channel data
        pulse.v_long_hi = read_iq_block(cpi_header.align_num_range_long_hi, cpi_header.num_range_long_hi, fid)
        pulse.v_long_lo = read_iq_block(cpi_header.align_num_range_long_lo, cpi_header.num_range_long_lo, fid)
        pulse.v_short_hi = read_iq_block(cpi_header.align_num_range_short_hi, cpi_header.num_range_short_hi, fid)
        pulse.v_short_lo = read_iq_block(cpi_header.align_num_range_short_lo, cpi_header.num_range_short_lo, fid)

        return pulse


    # CPI data header
    def read_cpi_header(fid, cpi_struct):
        tmp = fid.read(32 * 4)
        n = cpi_struct.unpack_from(tmp)
        if n[0] != 0x0055AAFF or n[1] != 0xFFAA5500:
            print('Error: Incorrect check codes: 0x{:08X} 0x{:08X}'.format(n[0], n[1]))
            return None

        cpi_header = Object()
        cpi_header.check_code_1 = n[0]
        cpi_header.check_code_2 = n[1]
        cpi_header.check_counter = n[2]
        cpi_header.data_size = n[3]
        cpi_header.count = n[4]
        cpi_header.num_samples = n[5]
        cpi_header.align_num_range_short_hi = n[6]
        cpi_header.align_num_range_short_lo = n[7]
        cpi_header.align_num_range_long_hi = n[8]
        cpi_header.align_num_range_long_lo = n[9]
        cpi_header.num_range_short_hi = n[10]
        cpi_header.num_range_short_lo = n[11]
        cpi_header.num_range_long_hi = n[12]
        cpi_header.num_range_long_lo = n[13]

        return cpi_header

    # PACK data header
    def read_pack_header(fid, pack_struct):
        tmp = fid.read(32 * 4)
        n = pack_struct.unpack_from(tmp)
        if n[0] != 0x55555555 or n[1] != 0xAAAAAAAA:
            print('Error: Incorrect check codes: 0x{:08X} 0x{:08X}'.format(n[0], n[1]))
            return None

        pack_header = Object()
        pack_header.check_code_1 = n[0]
        pack_header.check_code_2 = n[1]
        pack_header.check_counter = n[2]
        pack_header.num_cpi = n[3]
        pack_header.pack_data_size = n[4]
        pack_header.division_number = n[5]
        pack_header.division_part_number = n[6]
        pack_header.mode = n[7] & 0x2 >> 1

        return pack_header

    # File header
    def read_file_header(fid, head_struct):
        tmp = fid.read(24)
        n = head_struct.unpack_from(tmp)
        file_header = Object()
        file_header.check_code_1 = n[0]
        file_header.check_code_2 = n[1]
        file_header.unknown_1 = n[2]
        file_header.unknown_2 = n[3]
        file_header.pack_count = n[4]

        return file_header
    
    # Open the file
    fid = open(file, 'rb')
    file_header = read_file_header(fid, head_struct)
    pack_header = read_pack_header(fid, pack_struct)

    if verbose:
        print('File check code 1: 0x{:08X}'.format(file_header.check_code_1))
        print('File check code 2: 0x{:08X}'.format(file_header.check_code_2))
        print('Pack count: {}'.format(file_header.pack_count))
        print('')
        print('Pack check code 1: 0x{:X}'.format(pack_header.check_code_1))
        print('Pack check code 2: 0x{:X}'.format(pack_header.check_code_2))
        print('Pack check counter: {}'.format(pack_header.check_counter))
        print('Number of CPI: {}'.format(pack_header.num_cpi))
        print('Pack data size: {}'.format(pack_header.pack_data_size))
        print('Division number: {} / {}'.format(
            pack_header.division_number, pack_header.division_part_number))
        print('Observation mode: {}'.format(pack_header.mode))
        
    # Rewind back to just after the file header
    fid.seek(24, 0)

    rays = []
    for _ in range(file_header.pack_count):
        pack_header = read_pack_header(fid, pack_struct)
        if verbose:
            print('Number of CPI: {}'.format(pack_header.num_cpi))
        for _ in range(pack_header.num_cpi):
            cpi_header = read_cpi_header(fid, cpi_struct)
            if cpi_header:
                pulses = []
                for k in range(cpi_header.count):
                    pulse = read_pulse(fid, pri_struct, cpi_header)
                    pulses.append(pulse)
                rays.append(pulses)
                if verbose:
                    print('    cpi:{:3d}   E:{:.2f}   A:{:5.2f}-{:5.2f} ({})   size:{}'.format(
                        cpi_header.check_counter,
                        pulses[0].elevation, pulses[0].azimuth, pulses[-1].azimuth,
                        cpi_header.count,
                        cpi_header.data_size))
            else:
                break
    
    if filesize < fid.tell():
        print('Warning: There is still {} B left.'.format(filesize - fid.tell()))
    fid.close()
    
    return rays

In [ ]:
s = time.time()
ray_pulses = read_file(file)
e = time.time()
print('Data read in {:.2f} s'.format(e - s))

In [ ]:
print('Ray summary:')
for k, pulses in enumerate(ray_pulses):
    print('k:{:3d}    E:{:.2f}   A:{:6.2f}-{:6.2f} ({})'.format(
        k, pulses[0].elevation, pulses[0].azimuth, pulses[-1].azimuth, len(pulses)))

In [ ]:
fid = open(file, 'rb')
fid.seek(408, 0)
tmp = fid.read(16)
num = np.frombuffer(tmp, '<i2')
fid.close()
print(tmp)
print(num)
x = np.array(num[::2] + 1j * num[1::2], dtype=np.csingle)
print(x)
print(ray_pulses[0][0].h_long_hi[:4])

In [ ]:
naz = len(ray_pulses)
ngate = ray_pulses[0][0].ngate_long_hi
# ngate = ray_pulses[0][0].ngate_short_hi
z = np.zeros((naz, ngate), dtype=np.single)
v = np.zeros((naz, ngate), dtype=np.single)

for k, pulses in enumerate(ray_pulses):
    p = np.zeros((len(pulses), ngate), dtype=np.csingle)
    for j, pulse in enumerate(pulses):
        p[j, :] = pulse.v_long_hi
        #p[j, :] = pulse.h_short_hi
#     v[k, :] = np.angle(np.sum(p[1:, :] * np.conj(p[:-1, :]), axis=0))
    v[k, :] = np.angle(np.sum(np.multiply(p[1:, :], np.conj(p[:-1, :])), axis=0))
    z[k, :] = 20 * np.log10(np.sum(np.abs(p), axis=0)) - 60
v[z < -10] = np.nan

In [ ]:
plt.figure(figsize=(6, 8), dpi=200)
plt.subplot(2, 1, 1)
plt.imshow(z[:, :500].T)
plt.colorbar()

cmap = matplotlib.cm.RdYlGn
plt.subplot(2, 1, 2)
plt.imshow(v[:, :500].T, cmap=cmap, vmin=-np.pi, vmax=np.pi)
plt.colorbar()    

In [ ]:
plt.close()

In [ ]:
pulses = ray_pulses[20]
p = np.zeros((len(pulses), ngate), dtype=np.csingle)
for j, pulse in enumerate(pulses):
    p[j, :] = pulse.h_long_hi
g = 300

In [ ]:
for g in range(300, 305):
    plt.figure(dpi=200)
    plt.subplot(2, 1, 1)
    plt.plot(np.real(p[:, g]))
    plt.plot(np.imag(p[:, g]))
    
    f = np.fft.fft(p[:, g], 512)
    plt.subplot(2, 1, 2)
    plt.plot(20 * np.log10(np.abs(f)), 'g')
    plt.ylim((40, 60))

In [ ]:
for g in range(300, 310):
    print(g,
          np.angle(np.sum(np.multiply(p[1:, g], np.conj(p[:-1, g])), axis=0)),
          np.angle(np.sum(p[1:, g] * np.conj(p[:-1, g]), axis=0)))

In [ ]:
g = 300
for a in range(50, 53):
    pulses = ray_pulses[a]
    p = np.zeros((len(pulses), ngate), dtype=np.csingle)
    for j, pulse in enumerate(pulses):
        p[j, :] = pulse.h_long_hi

    plt.figure(dpi=200)
    plt.plot(np.real(p[:, g]))
    plt.plot(np.imag(p[:, g]))